In [76]:
#INPUT_FN = "input_test.txt" # 41
INPUT_FN = "input.txt"

with open(INPUT_FN) as f:
    lines = [l.rstrip() for l in f]

if lines[-1]=='':
    lines=lines[:,-1]

# parse map

In [77]:
import numpy as np

In [78]:
nrows = len(lines)
ncols = len(lines[0])

In [79]:
map_obst = np.zeros( (nrows, ncols), dtype=bool)

In [80]:
map_obst

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], shape=(130, 130))

parse map and guard direction

In [81]:
map_obst_list = []

for irow, l0 in enumerate(lines):
    for icol, c0 in enumerate(l0):
        if c0=="#":
            map_obst[irow,icol] = True
            map_obst_list.append((irow,icol))
        elif c0=="V":
            guard_pos = (irow,icol)
            guard_dir = (1,0)
        elif c0==">":
            guard_pos = (irow,icol)
            guard_dir = (0,1)
        elif c0=="^":
            guard_pos = (irow,icol)
            guard_dir = (-1,0)
        elif c0=="<":
            guard_pos = (irow,icol)
            guard_dir = (0,-1)

In [82]:
map_obst_list

[(0, 1),
 (0, 6),
 (0, 26),
 (0, 43),
 (0, 66),
 (0, 68),
 (0, 84),
 (0, 101),
 (0, 106),
 (0, 118),
 (0, 129),
 (1, 33),
 (1, 39),
 (1, 52),
 (1, 65),
 (1, 73),
 (1, 77),
 (1, 81),
 (1, 82),
 (1, 91),
 (1, 122),
 (2, 14),
 (2, 16),
 (2, 29),
 (2, 77),
 (2, 78),
 (2, 96),
 (2, 117),
 (2, 120),
 (3, 14),
 (3, 44),
 (3, 72),
 (3, 77),
 (3, 82),
 (3, 91),
 (3, 125),
 (3, 128),
 (4, 12),
 (4, 14),
 (4, 17),
 (4, 19),
 (4, 47),
 (4, 85),
 (4, 96),
 (4, 109),
 (5, 1),
 (5, 72),
 (5, 107),
 (5, 114),
 (5, 127),
 (5, 129),
 (6, 11),
 (6, 41),
 (6, 62),
 (6, 66),
 (6, 72),
 (6, 81),
 (6, 86),
 (6, 87),
 (6, 129),
 (7, 6),
 (7, 9),
 (7, 10),
 (7, 82),
 (7, 95),
 (8, 8),
 (8, 16),
 (8, 27),
 (8, 35),
 (8, 44),
 (8, 63),
 (8, 84),
 (8, 98),
 (8, 120),
 (9, 16),
 (9, 21),
 (9, 23),
 (9, 24),
 (9, 50),
 (9, 59),
 (9, 64),
 (9, 82),
 (10, 13),
 (10, 54),
 (10, 75),
 (10, 91),
 (10, 97),
 (10, 111),
 (10, 127),
 (11, 10),
 (11, 36),
 (11, 40),
 (11, 64),
 (11, 110),
 (11, 117),
 (12, 8),
 (12, 57),
 (

In [83]:
guard_pos

(58, 80)

In [84]:
guard_dir

(-1, 0)

Move the guard and mark areas

# 1st algorithm

In [49]:
map_visited = np.zeros_like(map_obst)
map_visited_next_dir = np.zeros( (*map_obst.shape,2) ) # index 0 for y and 1 for x
candidate_obstacles_for_loop = np.zeros_like(map_obst)

# initialize guard position
guard_pos0 = np.array(guard_pos)
guard_dir0 = np.array(guard_dir)
count_loop_candidates = 0

outside = False
while not outside:

    if map_visited[*guard_pos0]:
        print(f"Location {guard_pos0} was visited before. Checking if adding an obstacle can lead to loop")
        # This location was visited before.
        # Check what was the direction it headed to
        previous_dir = map_visited_next_dir[*guard_pos,:]
        print (f"previous visited guard direction {previous_dir}")
        
        #With the new direction, could I place an obstacle such that would make guard follow the previously used direction?
        if guard_dir0[0] == -previous_dir[1] and guard_dir0[1] == previous_dir[0]: # TODO: check rotation is right
            # A single obstacle would make guard go in a loop using previously used route
            # Where to place the obstacle? Right in front

            candidate_obstacle_placement = guard_pos0+guard_dir0 #element-wise addition
            print(f"potential obstacle placement at {candidate_obstacle_placement}")

            # check is outside
            if candidate_obstacle_placement[0]<0 or candidate_obstacle_placement[0]>=nrows or candidate_obstacle_placement[1]<0 or candidate_obstacle_placement[1]>=ncols:
                print(f"candidate_obstacle_placement is beyond boundaries")
            else:
                print(f"candidate_obstacle_placement is within boundaries")
                candidate_obstacles_for_loop[*candidate_obstacle_placement] = True


    map_visited[guard_pos0[0], guard_pos0[1]]=True
    print(f"{guard_pos0=}, {guard_dir0=}")

    # Move the guard
    for i in range(4):# rotations
        
        #new_guard_pos = (guard_pos0[0]+guard_dir0[0], guard_pos0[1]+guard_dir0[1])
        new_guard_pos = guard_pos0+guard_dir0

        #print(f"try move to position {new_guard_pos}")
        if new_guard_pos[0]<0 or new_guard_pos[0]>=nrows or new_guard_pos[1]<0 or new_guard_pos[1]>=ncols:
            print(f"Reached outside condition with, {new_guard_pos=}")
            outside=True # Reached outside, break
            break
        if map_obst[new_guard_pos[0],new_guard_pos[1]]:
            #Rotate dir and try again
            print(f"Obstacle at {new_guard_pos}, rotate")
            guard_dir0 = np.array( [guard_dir0[1] , -guard_dir0[0]])
        else:
            # can move forward
            # store direction
            map_visited_next_dir[*guard_pos0,:] = guard_dir0

            guard_pos0=new_guard_pos
            break

guard_pos0=array([6, 4]), guard_dir0=array([-1,  0])
guard_pos0=array([5, 4]), guard_dir0=array([-1,  0])
guard_pos0=array([4, 4]), guard_dir0=array([-1,  0])
guard_pos0=array([3, 4]), guard_dir0=array([-1,  0])
guard_pos0=array([2, 4]), guard_dir0=array([-1,  0])
guard_pos0=array([1, 4]), guard_dir0=array([-1,  0])
Obstacle at [0 4], rotate
guard_pos0=array([1, 5]), guard_dir0=array([0, 1])
guard_pos0=array([1, 6]), guard_dir0=array([0, 1])
guard_pos0=array([1, 7]), guard_dir0=array([0, 1])
guard_pos0=array([1, 8]), guard_dir0=array([0, 1])
Obstacle at [1 9], rotate
guard_pos0=array([2, 8]), guard_dir0=array([1, 0])
guard_pos0=array([3, 8]), guard_dir0=array([1, 0])
guard_pos0=array([4, 8]), guard_dir0=array([1, 0])
guard_pos0=array([5, 8]), guard_dir0=array([1, 0])
guard_pos0=array([6, 8]), guard_dir0=array([1, 0])
Obstacle at [7 8], rotate
guard_pos0=array([6, 7]), guard_dir0=array([ 0, -1])
guard_pos0=array([6, 6]), guard_dir0=array([ 0, -1])
guard_pos0=array([6, 5]), guard_dir0=ar

*This algorithm is not working correctly*

It assumes that the only places where obstacles can be placed that can lead to loops are next to used paths, which is not necessary the case.

# 2nd algorithm

Try creating a function that checks route provided initial map, visited locations including heading out direction.

In this way we can go back to one of the visited locations and replace it with an obstacle and then check for looping.

To check if it loops, it is probably more efficient to follow the route and register also out direction.
A loop will be, by following the route, if it encounters a location and direction that has already been used.

Rather than having a 2D, maybe is better to store visited places as a list of tuples (y,x, diry, dirx), and then check if it is list

In [85]:
map_visited_ext=[]

In [98]:
def check_route_is_loop(map_visited_ext_0: list, map_obst0, guard_pos0: tuple, guard_dir0:tuple):
    # returns a tuple (b_leads_to_loop, map_visited_ext_ret)
    # b_leads_to_loop True if a continuation of the guard movements leads to a loop

    global nrows, ncols

    #print(f"{guard_pos0=} , {guard_dir0=}")
    map_visited_ext = map_visited_ext_0 # makes a copy
    guard_pos = guard_pos0
    guard_dir = guard_dir0

    outside = False
    while not outside:
        
        cur_guard_status = (*guard_pos, *guard_dir)

        #check if current location has been visited
        if cur_guard_status in map_visited_ext:
            #print(f"Location and dir {cur_guard_status} was visited before. This means a loop is closed.")
            return True, map_visited_ext
        
        # Continuation means location and dir has never been used before
        map_visited_ext.append(cur_guard_status)
        #print(f"Visiting {cur_guard_status}")

        # Move the guard
        for i in range(4):# rotations

            candidate_guard_pos = (guard_pos[0]+guard_dir[0], guard_pos[1]+guard_dir[1] )

            if candidate_guard_pos[0]<0 or candidate_guard_pos[0]>=nrows or candidate_guard_pos[1]<0 or candidate_guard_pos[1]>=ncols:
                #print(f"Reached outside condition with, {candidate_guard_pos=}")
                return False, map_visited_ext # Finish exploration, No loop detected

            if candidate_guard_pos in map_obst0:
                #Rotate dir and try again
                #print(f"Obstacle at {candidate_guard_pos}. Rotate")
                guard_dir = [guard_dir[1] , -guard_dir[0]]
            else:
                # can move forward
                # store direction

                #cur_guard_status = (*candidate_guard_pos, *guard_dir)

                guard_pos=candidate_guard_pos
                break # no more rotations needed


Check if it works correctly as problem part 1

In [99]:
res = check_route_is_loop([], map_obst_list, guard_pos, guard_dir)

Note that there are some states that have same position but different directions.
To get the number of places visited we need to remove duplicates. Best is to use only the positions and create a set. then get the length of the set

In [100]:
poss_visited = { (loc[0], loc[1]) for loc in res[1] }

In [101]:
poss_visited

{(113, 26),
 (59, 55),
 (18, 17),
 (78, 68),
 (79, 33),
 (89, 77),
 (90, 42),
 (91, 7),
 (36, 71),
 (82, 38),
 (55, 84),
 (28, 67),
 (47, 80),
 (40, 41),
 (100, 92),
 (60, 19),
 (46, 121),
 (92, 88),
 (11, 14),
 (38, 117),
 (111, 101),
 (17, 94),
 (58, 95),
 (112, 66),
 (124, 40),
 (69, 104),
 (75, 35),
 (103, 17),
 (61, 100),
 (115, 71),
 (116, 36),
 (81, 78),
 (27, 107),
 (34, 3),
 (52, 51),
 (39, 81),
 (45, 12),
 (30, 112),
 (37, 8),
 (108, 68),
 (14, 24),
 (73, 110),
 (74, 75),
 (128, 46),
 (93, 88),
 (66, 71),
 (39, 117),
 (67, 36),
 (46, 13),
 (126, 85),
 (97, 58),
 (70, 41),
 (43, 87),
 (89, 54),
 (35, 83),
 (36, 48),
 (82, 15),
 (101, 28),
 (88, 95),
 (48, 22),
 (99, 104),
 (59, 68),
 (79, 46),
 (5, 60),
 (71, 42),
 (111, 78),
 (17, 71),
 (63, 38),
 (112, 43),
 (77, 85),
 (82, 51),
 (28, 80),
 (69, 81),
 (75, 12),
 (21, 41),
 (80, 90),
 (41, 19),
 (18, 115),
 (112, 79),
 (105, 40),
 (124, 53),
 (69, 117),
 (10, 31),
 (116, 49),
 (62, 78),
 (81, 91),
 (128, 23),
 (73, 87),
 (34,

In [102]:
len(poss_visited)

4890

OK

What we can do now is to reuse this visited locations and add an obstacle to each of them in turn and check if we get a loop.

In [103]:
count_loops = 0

for i, p0 in enumerate(poss_visited):
    print(f"Iteration {i+1}/{len(poss_visited)}, Trying obstacle at {p0}")
    map_obst_list_0 = map_obst_list.copy()

    map_obst_list_0.append( p0 )

    res = check_route_is_loop([], map_obst_list_0, guard_pos, guard_dir)
    
    if res[0]: count_loops+=1

Iteration 1/4890, Trying obstacle at (113, 26)
Iteration 2/4890, Trying obstacle at (59, 55)
Iteration 3/4890, Trying obstacle at (18, 17)
Iteration 4/4890, Trying obstacle at (78, 68)
Iteration 5/4890, Trying obstacle at (79, 33)
Iteration 6/4890, Trying obstacle at (89, 77)
Iteration 7/4890, Trying obstacle at (90, 42)
Iteration 8/4890, Trying obstacle at (91, 7)
Iteration 9/4890, Trying obstacle at (36, 71)
Iteration 10/4890, Trying obstacle at (82, 38)
Iteration 11/4890, Trying obstacle at (55, 84)
Iteration 12/4890, Trying obstacle at (28, 67)
Iteration 13/4890, Trying obstacle at (47, 80)
Iteration 14/4890, Trying obstacle at (40, 41)
Iteration 15/4890, Trying obstacle at (100, 92)
Iteration 16/4890, Trying obstacle at (60, 19)
Iteration 17/4890, Trying obstacle at (46, 121)
Iteration 18/4890, Trying obstacle at (92, 88)
Iteration 19/4890, Trying obstacle at (11, 14)
Iteration 20/4890, Trying obstacle at (38, 117)
Iteration 21/4890, Trying obstacle at (111, 101)
Iteration 22/4890

In [104]:
count_loops

1995

OK